<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_14_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url_1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-1.csv'
url_2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-2.csv'
url_3 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-3.csv'
url_4 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-4.csv'
url_5 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-5.csv'

data_1 = pd.read_csv(url_1)
data_2 = pd.read_csv(url_2)
data_3 = pd.read_csv(url_3)
data_4 = pd.read_csv(url_4)
data_5 = pd.read_csv(url_5)

data = data_1
data = data.append(data_2)
data = data.append(data_3)
data = data.append(data_4)
data = data.append(data_5)

data = data.reset_index()
data = data.drop(labels=['index'], axis=1)

In [55]:
# [1] KNN, KNN(predict_proba)

from sklearn.neighbors import KNeighborsClassifier
import math

# selet : class == null & Amount > 200
NC = data[data.Class.isna()]
NC = NC[NC.Amount > 200]

# perform KNN (prepare to classify for new samples)
X_cols = data.columns[1:-2]
VC = data[~data.Class.isna()]  # all data with valid Class values
knn = KNeighborsClassifier(n_neighbors=5).fit(VC[X_cols], VC.Class)

# predict for null data
pred = knn.predict_proba(NC[X_cols])
pred = pd.DataFrame(pred, columns=['neg', 'pos'])
pred = pred.assign(esti=pred.pos.apply(lambda x: 1 if x > 0 else 0))

# merge pred with NC
NC.reset_index(inplace=True)
NC = pd.merge(NC, pred, left_index=True, right_index=True)
NC = NC.assign(Class=NC.esti)
NC = NC.drop(labels=['neg', 'pos', 'esti'], axis=1)
NC = NC.set_index(keys='index', drop=True)

# merge NC and VC
base = pd.concat([VC, NC], axis=0)

# find range
maxA = base[base.Class == 1].Amount.max()
minA = base[base.Class == 1].Amount.min()

print(f'Answer [1] : {math.floor(maxA - minA)}')

Answer [1] : 25691


In [56]:
# [2] ttest_ind

from scipy.stats import ttest_ind

class0 = base[base.Class == 0].V2
class1 = base[base.Class == 1].V2

t_val, p_val = ttest_ind(class0, class1, equal_var=False)

print(f'{math.floor(t_val)}')

22


In [57]:
# [3] LinearRegression (R-Squared)

from sklearn.linear_model import LinearRegression

train_X = base[base.Class == 1][X_cols]
train_y = base[base.Class == 1].Amount

lr = LinearRegression().fit(train_X, train_y)
r_squared = lr.score(train_X, train_y)

print(f'Answer [3] : {math.floor(r_squared * 100) / 100}')

Answer [3] : 0.93


In [58]:
# [4] LogisticRegression

from sklearn.linear_model import LogisticRegression

# create train and test data
train_X = base[base.Time <= 100000][list(X_cols) + ['Amount']]
train_y = base[base.Time <= 100000].Class
test_X = base[base.Time > 100000][list(X_cols) + ['Amount']]
test_y = base[base.Time > 100000].Class

# train
lr = LogisticRegression(C=100000, random_state=1234, solver='newton-cg')
model = lr.fit(train_X, train_y)

# test
pred = model.predict_proba(test_X)
pred = pd.DataFrame(pred, columns=['neg', 'pos'])
pred = pred.assign(esti=pred.pos.apply(lambda x: 1 if x >= 0.4 else 0))

# merge estimation and fact
test_y = pd.DataFrame(test_y).reset_index()
pred = pd.merge(pred, test_y, left_index=True, right_index=True)

# calculate accuracy
pred = pred.assign(true=pred.apply(lambda x: 1 if x.esti == x.Class else 0,
                                   axis=1))
accuracy = pred.true.sum() / pred.true.count()

print(f'Answer [4] : {math.floor(accuracy * 1000) / 1000}')

Answer [4] : 0.815
